___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="CLRSWY"></p>

___

<h1><p style="text-align: center;">Data Analysis with Python <br>Project - 1</p><h1> - Traffic Police Stops <img src="https://docs.google.com/uc?id=17CPCwi3_VvzcS87TOsh4_U8eExOhL6Ki" class="img-fluid" alt="CLRSWY" width="200" height="100"> 

Does the ``gender`` of a driver have an impact on police behavior during a traffic stop? **In this chapter**, you will explore that question while practicing filtering, grouping, method chaining, Boolean math, string methods, and more!

***

## Examining traffic violations

Before comparing the violations being committed by each gender, you should examine the ``violations`` committed by all drivers to get a baseline understanding of the data.

In this exercise, you'll count the unique values in the ``violation`` column, and then separately express those counts as proportions.

> Before starting your work in this section **repeat the steps which you did in the previos chapter for preparing the data.** Continue to this chapter based on where you were in the end of the previous chapter.

In [1]:
%run ./01PreparingTheDataForAnalysis.ipynb

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**INSTRUCTIONS**

*   Count the unique values in the ``violation`` column, to see what violations are being committed by all drivers.
*   Express the violation counts as proportions of the total.

In [2]:
#Count the unique values in the violation column, 
#to see what violations are being committed by all drivers.

ri.head()

,id,stop_date,stop_time,location_raw,county_fips,fine_grained_location,police_department,driver_gender,driver_age_raw,driver_age,...,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district,combined
stop_datetime,,,,,,,,,,,,,,,,,,,,,
2005-01-02 01:55:00,RI-2005-00001,2005-01-02,01:55,Zone K1,NaN,NaN,600,M,1985.0,20.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K1,2005-01-02 01:55
2005-01-02 20:30:00,RI-2005-00002,2005-01-02,20:30,Zone X4,NaN,NaN,500,M,1987.0,18.0,...,NaN,NaN,False,Citation,False,16-30 Min,False,False,Zone X4,2005-01-02 20:30
2005-01-04 12:55:00,RI-2005-00004,2005-01-04,12:55,Zone X4,NaN,NaN,500,M,1986.0,19.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4,2005-01-04 12:55
2005-01-06 01:30:00,RI-2005-00005,2005-01-06,01:30,Zone X4,NaN,NaN,500,M,1978.0,27.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4,2005-01-06 01:30
2005-01-12 08:05:00,RI-2005-00006,2005-01-12,08:05,Zone X1,NaN,NaN,0,M,1973.0,32.0,...,NaN,NaN,False,Citation,False,30+ Min,True,False,Zone X1,2005-01-12 08:05


In [3]:
ri['violation'].value_counts()   # Count the unique values in the violation column.

Speeding               36111
Moving violation        6522
Equipment               3022
Registration/plates     1463
Other                    892
Name: violation, dtype: int64

In [4]:
a = ri["violation"].value_counts()    # Express the violation counts as proportions of the total
b = (a/a.sum()) * 100
b

Speeding               75.215580
Moving violation       13.584670
Equipment               6.294522
Registration/plates     3.047282
Other                   1.857946
Name: violation, dtype: float64

In [5]:
ri.violation.value_counts(normalize=True) # Joseph hoca

Speeding               0.752156
Moving violation       0.135847
Equipment              0.062945
Registration/plates    0.030473
Other                  0.018579
Name: violation, dtype: float64

In [6]:
ri["violation"].value_counts() / ri["violation"].value_counts().sum() * 100 

# Express the violation counts as proportions of the total

Speeding               75.215580
Moving violation       13.584670
Equipment               6.294522
Registration/plates     3.047282
Other                   1.857946
Name: violation, dtype: float64

In [47]:
def calculate_value_counts(df, i):
    percent = df[i].value_counts()*100/df.shape[0]
    piece = df[i].value_counts()
    return pd.DataFrame(list(zip(piece, percent)), columns = ["piece", "percent(%)"],index=  df[i].value_counts().keys())

In [49]:
calculate_value_counts(ri, "violation")

,piece,percent(%)
Speeding,36111,75.215580
Moving violation,6522,13.584670
Equipment,3022,6.294522
Registration/plates,1463,3.047282
Other,892,1.857946


***

## Comparing violations by gender

The question we're trying to answer is whether male and female drivers tend to commit different types of traffic violations.

You'll first create a ``DataFrame`` for each gender, and then analyze the ``violations`` in each ``DataFrame`` separately.

**INSTRUCTIONS**

*   Create a ``DataFrame``, female, that only contains rows in which ``driver_gender`` is ``'F'``.
*   Create a ``DataFrame``, male, that only contains rows in which ``driver_gender`` is ``'M'``.
*   Count the ``violations`` committed by female drivers and express them as proportions.
*   Count the violations committed by male drivers and express them as proportions.

In [8]:
#Create a DataFrame, female, that only contains rows in which driver_gender is 'F'.
#Create a DataFrame, male, that only contains rows in which driver_gender is 'M'

df_female = ri[ri["driver_gender"] == "F"]
df_male = ri[ri["driver_gender"] == "M"]
df_female


,id,stop_date,stop_time,location_raw,county_fips,fine_grained_location,police_department,driver_gender,driver_age_raw,driver_age,...,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district,combined
stop_datetime,,,,,,,,,,,,,,,,,,,,,
2005-02-24 01:20:00,RI-2005-00016,2005-02-24,01:20,Zone X3,NaN,NaN,200,F,1983.0,22.0,...,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone X3,2005-02-24 01:20
2005-03-14 10:00:00,RI-2005-00019,2005-03-14,10:00,Zone K3,NaN,NaN,300,F,1984.0,21.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K3,2005-03-14 10:00
2005-03-29 23:20:00,RI-2005-00026,2005-03-29,23:20,Zone K3,NaN,NaN,300,F,1971.0,34.0,...,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K3,2005-03-29 23:20
2005-06-06 13:20:00,RI-2005-00035,2005-06-06,13:20,Zone X4,NaN,NaN,500,F,1986.0,19.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4,2005-06-06 13:20
2005-06-18 16:30:00,RI-2005-00037,2005-06-18,16:30,Zone X4,NaN,NaN,500,F,1964.0,41.0,...,NaN,NaN,False,Arrest Driver,True,30+ Min,False,False,Zone X4,2005-06-18 16:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-08-08 22:20:00,RI-2006-35911,2006-08-08,22:20,Zone K3,NaN,NaN,300,F,1980.0,26.0,...,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K3,2006-08-08 22:20
2006-08-08 22:25:00,RI-2006-35913,2006-08-08,22:25,Zone K3,NaN,NaN,300,F,1987.0,19.0,...,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K3,2006-08-08 22:25
2006-08-08 22:45:00,RI-2006-35916,2006-08-08,22:45,Zone K1,NaN,NaN,600,F,1978.0,28.0,...,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K1,2006-08-08 22:45


In [9]:
ri["driver_gender"].value_counts(dropna=False) # Joseph hoca

M    34701
F    13309
Name: driver_gender, dtype: int64

In [10]:
df_male

,id,stop_date,stop_time,location_raw,county_fips,fine_grained_location,police_department,driver_gender,driver_age_raw,driver_age,...,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district,combined
stop_datetime,,,,,,,,,,,,,,,,,,,,,
2005-01-02 01:55:00,RI-2005-00001,2005-01-02,01:55,Zone K1,NaN,NaN,600,M,1985.0,20.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K1,2005-01-02 01:55
2005-01-02 20:30:00,RI-2005-00002,2005-01-02,20:30,Zone X4,NaN,NaN,500,M,1987.0,18.0,...,NaN,NaN,False,Citation,False,16-30 Min,False,False,Zone X4,2005-01-02 20:30
2005-01-04 12:55:00,RI-2005-00004,2005-01-04,12:55,Zone X4,NaN,NaN,500,M,1986.0,19.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4,2005-01-04 12:55
2005-01-06 01:30:00,RI-2005-00005,2005-01-06,01:30,Zone X4,NaN,NaN,500,M,1978.0,27.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4,2005-01-06 01:30
2005-01-12 08:05:00,RI-2005-00006,2005-01-12,08:05,Zone X1,NaN,NaN,0,M,1973.0,32.0,...,NaN,NaN,False,Citation,False,30+ Min,True,False,Zone X1,2005-01-12 08:05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-08-08 22:30:00,RI-2006-35914,2006-08-08,22:30,Zone K1,NaN,NaN,600,M,1967.0,39.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K1,2006-08-08 22:30
2006-08-08 22:30:00,RI-2006-35915,2006-08-08,22:30,Zone K2,NaN,NaN,900,M,1982.0,24.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K2,2006-08-08 22:30
2006-08-08 22:45:00,RI-2006-35917,2006-08-08,22:45,Zone K3,NaN,NaN,300,M,1973.0,33.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K3,2006-08-08 22:45


In [11]:
df_female["violation"].value_counts()

Speeding               10796
Moving violation        1318
Equipment                607
Registration/plates      367
Other                    221
Name: violation, dtype: int64

In [12]:
df_female["violation"].value_counts() / df_female["violation"].value_counts().sum() * 100 

Speeding               81.118040
Moving violation        9.903073
Equipment               4.560824
Registration/plates     2.757532
Other                   1.660530
Name: violation, dtype: float64

In [13]:
df_female["violation"].value_counts(normalize = True) #joseph hoca

Speeding               0.811180
Moving violation       0.099031
Equipment              0.045608
Registration/plates    0.027575
Other                  0.016605
Name: violation, dtype: float64

In [14]:
df_male["violation"].value_counts() / df_male["violation"].value_counts().sum() * 100 

Speeding               72.951788
Moving violation       14.996686
Equipment               6.959454
Registration/plates     3.158410
Other                   1.933662
Name: violation, dtype: float64

In [15]:
df_male["violation"].value_counts(normalize = True) # Joseph hoca

Speeding               0.729518
Moving violation       0.149967
Equipment              0.069595
Registration/plates    0.031584
Other                  0.019337
Name: violation, dtype: float64

In [16]:
ri.groupby(["driver_gender"])["violation"].value_counts(normalize=True, dropna=False)

driver_gender  violation          
F              Speeding               0.811180
               Moving violation       0.099031
               Equipment              0.045608
               Registration/plates    0.027575
               Other                  0.016605
M              Speeding               0.729518
               Moving violation       0.149967
               Equipment              0.069595
               Registration/plates    0.031584
               Other                  0.019337
Name: violation, dtype: float64

***

## Comparing speeding outcomes by gender

When a driver is pulled over for speeding, many people believe that gender has an impact on whether the driver will receive a ticket or a warning. Can you find evidence of this in the dataset?

First, you'll create two ``DataFrames`` of drivers who were stopped for ``speeding``: one containing ***females*** and the other containing ***males***.

Then, for each **gender**, you'll use the ``stop_outcome`` column to calculate what percentage of stops resulted in a ``"Citation"`` (meaning a ticket) versus a ``"Warning"``.

**INSTRUCTIONS**

*   Create a ``DataFrame``, ``female_and_speeding``, that only includes female drivers who were stopped for speeding.
*   Create a ``DataFrame``, ``male_and_speeding``, that only includes male drivers who were stopped for speeding.
*   Count the **stop outcomes** for the female drivers and express them as proportions.
*   Count the **stop outcomes** for the male drivers and express them as proportions.

In [17]:
ri.shape

(48010, 25)

In [18]:
df_female_and_speeding = ri[ri["driver_gender"]=="F"]
df_female_and_speeding = df_female_and_speeding[df_female_and_speeding["violation"]=="Speeding"]

In [19]:
#df_female_and_speeding = ri[(ri["driver_gender"]=="F") & (ri["violation"]=="Speeding")]

In [20]:
df_female_and_speeding.shape

(10796, 25)

In [21]:
df_female_and_speeding[df_female_and_speeding["violation"]=="Speeding"].columns

Index(['id', 'stop_date', 'stop_time', 'location_raw', 'county_fips',
       'fine_grained_location', 'police_department', 'driver_gender',
       'driver_age_raw', 'driver_age', 'driver_race_raw', 'driver_race',
       'violation_raw', 'violation', 'search_conducted', 'search_type_raw',
       'search_type', 'contraband_found', 'stop_outcome', 'is_arrested',
       'stop_duration', 'out_of_state', 'drugs_related_stop', 'district',
       'combined'],
      dtype='object')

In [22]:
df_male_and_speeding = ri[ri["driver_gender"]=="M"]
df_male_and_speeding = df_male_and_speeding[df_male_and_speeding["violation"]=="Speeding"]

In [23]:
df_female_and_speeding["stop_outcome"].value_counts() / df_female_and_speeding["stop_outcome"].value_counts().sum() * 100 

Citation            97.341608
Warning              1.296777
Arrest Driver        0.741015
N/D                  0.361245
Arrest Passenger     0.231567
No Action            0.027788
Name: stop_outcome, dtype: float64

In [24]:
df_male_and_speeding["stop_outcome"].value_counts() / df_male_and_speeding["stop_outcome"].value_counts().sum() * 100 

Citation            95.729804
Arrest Driver        2.622951
Warning              1.066561
N/D                  0.339720
Arrest Passenger     0.201462
No Action            0.039502
Name: stop_outcome, dtype: float64

***

## Calculating the search rate

During a traffic stop, the police officer sometimes conducts a search of the vehicle. In this exercise, you'll calculate the percentage of all stops that result in a vehicle search, also known as the **search rate**.

**INSTRUCTIONS**

*   Check the data type of ``search_conducted`` to confirm that it's a ``Boolean Series``.
*   Calculate the search rate by counting the ``Series`` values and expressing them as proportions.
*   Calculate the search rate by taking the mean of the ``Series``. (It should match the proportion of ``True`` values calculated above.)

In [25]:
ri[["search_conducted"]].dtypes

search_conducted    object
dtype: object

In [50]:
ri = ri.astype ({"search_conducted": "bool"})

In [27]:
ri[["search_conducted"]].dtypes

search_conducted    bool
dtype: object

In [28]:
ri["search_conducted"].value_counts() / ri["search_conducted"].value_counts().sum() * 100

False    95.809206
True      4.190794
Name: search_conducted, dtype: float64

In [29]:
ri["search_conducted"].mean() * 100

4.19079358466986

***

## Comparing search rates by gender

You'll compare the rates at which **female** and **male** drivers are searched during a traffic stop. Remember that the vehicle search rate across all stops is about **3.8%**.

First, you'll filter the ``DataFrame`` by gender and calculate the search rate for each group separately. Then, you'll perform the same calculation for both genders at once using a ``.groupby()``.

**INSTRUCTIONS 1/3**

*   Filter the ``DataFrame`` to only include **female** drivers, and then calculate the search rate by taking the mean of ``search_conducted``.

In [30]:
df_female

,id,stop_date,stop_time,location_raw,county_fips,fine_grained_location,police_department,driver_gender,driver_age_raw,driver_age,...,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district,combined
stop_datetime,,,,,,,,,,,,,,,,,,,,,
2005-02-24 01:20:00,RI-2005-00016,2005-02-24,01:20,Zone X3,NaN,NaN,200,F,1983.0,22.0,...,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone X3,2005-02-24 01:20
2005-03-14 10:00:00,RI-2005-00019,2005-03-14,10:00,Zone K3,NaN,NaN,300,F,1984.0,21.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K3,2005-03-14 10:00
2005-03-29 23:20:00,RI-2005-00026,2005-03-29,23:20,Zone K3,NaN,NaN,300,F,1971.0,34.0,...,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K3,2005-03-29 23:20
2005-06-06 13:20:00,RI-2005-00035,2005-06-06,13:20,Zone X4,NaN,NaN,500,F,1986.0,19.0,...,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4,2005-06-06 13:20
2005-06-18 16:30:00,RI-2005-00037,2005-06-18,16:30,Zone X4,NaN,NaN,500,F,1964.0,41.0,...,NaN,NaN,False,Arrest Driver,True,30+ Min,False,False,Zone X4,2005-06-18 16:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006-08-08 22:20:00,RI-2006-35911,2006-08-08,22:20,Zone K3,NaN,NaN,300,F,1980.0,26.0,...,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K3,2006-08-08 22:20
2006-08-08 22:25:00,RI-2006-35913,2006-08-08,22:25,Zone K3,NaN,NaN,300,F,1987.0,19.0,...,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K3,2006-08-08 22:25
2006-08-08 22:45:00,RI-2006-35916,2006-08-08,22:45,Zone K1,NaN,NaN,600,F,1978.0,28.0,...,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K1,2006-08-08 22:45


In [31]:
df_female["search_conducted"].mean() * 100

1.7807498685100307

**INSTRUCTIONS 2/3**

*   Filter the ``DataFrame`` to only include **male** drivers, and then repeat the search rate calculation.

In [32]:
df_male["search_conducted"].mean() * 100

5.115126365234431

**INSTRUCTIONS 3/3**

*   Group by driver gender to calculate the search rate for both groups simultaneously. (It should match the previous results.)

In [53]:
ri.groupby(["driver_gender"])[["search_conducted"]].mean()*100

,search_conducted
driver_gender,
F,1.780750
M,5.115126


In [33]:
ri.groupby(["driver_gender"]).mean()[["search_conducted"]]*100

,search_conducted
driver_gender,
F,1.780750
M,5.115126


In [34]:
ri.groupby(["driver_gender"]).mean()[["search_conducted"]]*100

,search_conducted
driver_gender,
F,1.780750
M,5.115126


***

## Adding a second factor to the analysis

Even though the search rate for males is much higher than for females, it's possible that the difference is mostly due to a second factor.

For example, you might hypothesize that the search rate varies by violation type, and the difference in search rate between males and females is because they tend to commit different violations.

You can test this hypothesis by examining the search rate for each combination of gender and violation. If the hypothesis was true, you would find that males and females are searched at about the same rate for each violation. Find out below if that's the case!

**INSTRUCTIONS 1/2**

*   Use a ``.groupby()`` to calculate the search rate for each combination of gender and violation. Are males and females searched at about the same rate for each violation?

In [35]:
ri.groupby(['driver_gender', 'violation'])[["search_conducted"]].mean()*100

search_conducted
driver_gender violation                            
F             Equipment                    7.907743
              Moving violation             4.779970
              Other                        4.524887
              Registration/plates         11.444142
              Speeding                     0.685439
M             Equipment                   12.339545
              Moving violation             8.877786
              Other                       15.499255
              Registration/plates         17.153285
              Speeding                     2.856014

**INSTRUCTIONS 2/2**

*   Reverse the ordering to group by violation before gender. The results may be easier to compare when presented this way.

In [36]:
ri.groupby(['violation', 'driver_gender'])[["search_conducted"]].mean()*100

search_conducted
violation           driver_gender                  
Equipment           F                      7.907743
                    M                     12.339545
Moving violation    F                      4.779970
                    M                      8.877786
Other               F                      4.524887
                    M                     15.499255
Registration/plates F                     11.444142
                    M                     17.153285
Speeding            F                      0.685439
                    M                      2.856014

***

## Counting protective frisks

During a vehicle search, the police officer may pat down the driver to check if they have a weapon. This is known as a ``"protective frisk."``

You'll first check to see how many times "Protective Frisk" was the only search type. Then, you'll use a string method to locate all instances in which the driver was frisked.

**INSTRUCTIONS**

*   Count the ``search_type`` values to see how many times ``"Protective Frisk"`` was the only search type.
*   Create a new column, frisk, that is ``True`` if ``search_type`` contains the string ``"Protective Frisk"`` and ``False`` otherwise.
*   Check the data type of frisk to confirm that it's a ``Boolean Series``.
*   Take the sum of frisk to count the total number of frisks.

In [37]:
ri.search_type.value_counts()

Incident to Arrest                                          958
Probable Cause                                              244
Protective Frisk                                            204
Inventory                                                   117
Incident to Arrest,Inventory                                116
Incident to Arrest,Probable Cause                            76
Incident to Arrest,Protective Frisk                          63
Reasonable Suspicion                                         43
Probable Cause,Protective Frisk                              36
Incident to Arrest,Inventory,Protective Frisk                33
Inventory,Protective Frisk                                   23
Incident to Arrest,Probable Cause,Protective Frisk           20
Incident to Arrest,Inventory,Probable Cause                  19
Protective Frisk,Reasonable Suspicion                        16
Inventory,Probable Cause                                     16
Probable Cause,Reasonable Suspicion     

In [38]:
ri[ri["search_type"]=="Protective Frisk"]["search_type"].value_counts()

Protective Frisk    204
Name: search_type, dtype: int64

In [39]:
ri.loc[ri["search_type"]=="Protective Frisk", "frisk"] = True
ri.loc[ri["search_type"]!="Protective Frisk", "frisk"] = False
a = ri[["search_type", "frisk"]].dropna()
a[a["search_type" ] == "Protective Frisk"]

,search_type,frisk
stop_datetime,,
2005-10-06 21:35:00,Protective Frisk,True
2005-10-07 11:33:00,Protective Frisk,True
2005-10-09 19:20:00,Protective Frisk,True
2005-10-10 07:00:00,Protective Frisk,True
2005-10-12 20:30:00,Protective Frisk,True
...,...,...
2006-08-03 02:15:00,Protective Frisk,True
2006-08-04 00:43:00,Protective Frisk,True
2006-08-04 10:05:00,Protective Frisk,True


In [56]:
#ri.loc[ri["search_type"]=="Protective Frisk", "frisk"] = True
#ri.loc[ri["search_type"]!="Protective Frisk", "frisk"] = False

ri['frisk'] = ri.search_type.str.contains('Protective Frisk', na = False)

In [58]:
ri.tail()

,id,stop_date,stop_time,location_raw,county_fips,fine_grained_location,police_department,driver_gender,driver_age_raw,driver_age,...,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district,combined,frisk
stop_datetime,,,,,,,,,,,,,,,,,,,,,
2006-08-08 22:45:00,RI-2006-35917,2006-08-08,22:45,Zone K3,NaN,NaN,300,M,1973.0,33.0,...,NaN,False,Citation,False,0-15 Min,False,False,Zone K3,2006-08-08 22:45,False
2006-08-08 22:45:00,RI-2006-35918,2006-08-08,22:45,Zone K3,NaN,NaN,300,F,1971.0,35.0,...,NaN,False,Citation,False,0-15 Min,True,False,Zone K3,2006-08-08 22:45,False
2006-08-08 22:53:00,RI-2006-35919,2006-08-08,22:53,Zone X4,NaN,NaN,500,M,1952.0,54.0,...,NaN,False,Citation,False,16-30 Min,True,False,Zone X4,2006-08-08 22:53,False
2006-08-08 23:00:00,RI-2006-35920,2006-08-08,23:00,Zone K1,NaN,NaN,600,F,1982.0,24.0,...,NaN,False,Citation,False,0-15 Min,False,False,Zone K1,2006-08-08 23:00,False
2006-08-08 23:00:00,RI-2006-35921,2006-08-08,23:00,Zone K3,NaN,NaN,300,M,1965.0,41.0,...,NaN,False,Citation,False,16-30 Min,False,False,Zone K3,2006-08-08 23:00,False


In [40]:
ri = ri.astype({"frisk":"bool"})
ri[["frisk"]].dtypes

frisk    bool
dtype: object

In [41]:
ri["frisk"].sum()

204

***

## Comparing frisk rates by gender

You'll compare the rates at which female and male drivers are frisked during a search. Are males frisked more often than females, perhaps because police officers consider them to be higher risk?

Before doing any calculations, it's important to filter the ``DataFrame`` to only include the relevant subset of data, namely stops in which a search was conducted.

**INSTRUCTIONS**

*   Create a ``DataFrame``, searched, that only contains rows in which ``search_conducted`` is ``True``.
*   Take the mean of the frisk column to find out what percentage of searches included a frisk.
*   Calculate the frisk rate for each gender using a ``.groupby()``.

In [42]:
df_search_conducted = ri[ri["search_conducted"] == True]

In [63]:
ri.groupby("driver_gender").frisk.mean()

driver_gender
F    0.00293
M    0.01049
Name: frisk, dtype: float64

In [62]:
df_search_conducted["frisk"].mean()

0.10139165009940358

In [45]:
df_search_conducted["frisk"].value_counts()

False    1808
True      204
Name: frisk, dtype: int64

In [46]:
df_search_conducted.groupby('driver_gender').frisk.mean()

driver_gender
F    0.101266
M    0.101408
Name: frisk, dtype: float64